In [320]:
# Nika's notebook

# Import statements
import numpy as np
import pandas as pd
from pathlib import Path
%matplotlib inline

In [321]:
# read the uscities csv. this file will be used for the longitude and latitude, as well as the density stats.
longlatdensity_csv = Path("../Data/uscities.csv")
longlatdensity_df = pd.read_csv(longlatdensity_csv)
longlatdensity_df.head()

#getting first zipcode from list of zipcodes and create primary MSA from stateid and city
longlatdensity_df[['firstzip']] = longlatdensity_df['zips'].str.split(",", n = 1, expand=True)
longlatdensity_df[['firstzip']] = longlatdensity_df['zips'].str[:5]
longlatdensity_df[['Primary MSA']]= longlatdensity_df['city']+"_"+longlatdensity_df['state_id']

#dropping unnecessary columns, reording columns and setting index
longlatdensity_df.drop(columns=['source','military','incorporated','timezone','city_ascii',"zips","id","county_fips","county_name","state_name","city","state_id"],inplace=True)
longlatdensity_df = longlatdensity_df[["Primary MSA", "lat", "lng","population","density","ranking","firstzip"]]
longlatdensity_df.drop_duplicates()
longlatdensity_df=longlatdensity_df.set_index("Primary MSA")
longlatdensity_df.head()

,lat,lng,population,density,ranking,firstzip
Primary MSA,,,,,,
New York_NY,40.6943,-73.9249,18713220.0,10715.0,1,11229
Los Angeles_CA,34.1139,-118.4068,12750807.0,3276.0,1,90291
Chicago_IL,41.8373,-87.6862,8604203.0,4574.0,1,60018
Miami_FL,25.7839,-80.2102,6445545.0,5019.0,1,33129
Dallas_TX,32.7936,-96.7662,5743938.0,1526.0,1,75287


In [322]:
# Using median income CSV files from EH notebook. 
# data cleanup using EH notebook. Differences are: (created new medican_income_affdf for affordability df calc)

median_income_csv = Path("../Data/household_income_msa_2019.csv")
median_income_df = pd.read_csv(median_income_csv)
median_income_df = median_income_df.transpose() # Transpose the column/headers
new_header = median_income_df.iloc[0] #grab the first row for the header
median_income_df = median_income_df[1:] #take the data less the header row
median_income_df.columns = new_header #set the header row as the df header
median_income_df = median_income_df[['Median income (dollars)','Mean income (dollars)']] # Use only median and mean income data columns
median_income_df.reset_index(inplace=True)
median_income_df[['MSA','extra']] = median_income_df['index'].str.split(",", n = 1, expand=True) # Split MSA from text string
median_income_df[['State','extra2']] = median_income_df['extra'].str.split(n = 1, expand=True) # Split State from remaining text string
median_income_df[['Primary MSA1','Secondary MSA']] = median_income_df['MSA'].str.split("-", n = 1, expand=True) # Split Primary MSA from first text string
median_income_df[['Primary State','Secondary State']] = median_income_df['State'].str.split("-", n = 1, expand=True) # Split Primary MSA from first text string
median_income_df.drop(columns=['index','extra','extra2','MSA','State'],inplace=True) # Drop unnecessary columns
median_income_df.drop_duplicates(subset=['Primary MSA1','Primary State'], inplace=True) # Drop duplicates -- first instance is the median and mean income by MSA
median_income_df.head()

median_income_df[['Primary MSA']]= median_income_df['Primary MSA1']+"_"+median_income_df['Primary State']
median_income_df = median_income_df[["Primary MSA", "Median income (dollars)", "Mean income (dollars)"]]
median_income_df.head()

#creating filtered dataframe for affordability calc

median_income_affdf=median_income_df.filter(['Primary MSA','Median income (dollars)','Mean income (dollars)'], axis=1)
median_income_affdf=median_income_affdf.set_index("Primary MSA")
median_income_affdf


Label,Median income (dollars),Mean income (dollars)
Primary MSA,,
Aberdeen_WA,"61,026","77,948"
Abilene_TX,"54,808","71,517"
Adrian_MI,"53,865","69,443"
Aguadilla_PR,"16,311","25,781"
Akron_OH,"57,158","80,800"
Alamogordo_NM,"39,371","54,602"
Albany_GA,"40,625","57,777"
Albany_OR,"61,488","74,582"
Albany_NY,"73,398","92,433"


In [339]:
# Getting 2019 rental prices to compare to the 2019 median income prices from above

#Data cleanup using AH notebook. 
#Differences are: (1.used the same EH primary MSA split from City,  2. did not transpose data, 3. created newdf for affordability calc)

csv_path_2 = Path("../Data/rental_prices_msa.csv")
rental_price_2019 = pd.read_csv(csv_path_2, usecols=['RegionName','2019-01',
 '2019-02',
 '2019-03',
 '2019-04',
 '2019-05',
 '2019-06',
 '2019-07',
 '2019-08',
 '2019-09',
 '2019-10',
 '2019-11',
 '2019-12',]) # used rental_price_raw.columns.tolist() to get the list of 2019 columns


#splitting City and State
rental_price_2019[['City','State']] = rental_price_2019.RegionName.str.split(",",expand=True)
rental_price_2019[['State']] = rental_price_2019['State'].str.strip()
rental_price_2019[['Primary MSA1','Secondary MSA']] = rental_price_2019['City'].str.split("-",n=1,expand=True)
rental_price_2019[['Primary MSA']]= rental_price_2019['Primary MSA1']+"_"+rental_price_2019['State']
rental_price_2019.drop(["RegionName","City","State","Primary MSA1", "Secondary MSA"], axis = 1, inplace = True)
rental_price_2019['Annual_Rent']= rental_price_2019.iloc[:, :-1].sum(axis=1)
rental_price_2019

#pd.set_option('display.max_rows', None)

#creating new to use for affordability calc

rental_price_2019_affdf=rental_price_2019.filter(['Primary MSA','Annual_Rent'], axis=1)
rental_price_2019_affdf.dropna(subset = ["Primary MSA"], inplace=True)

#rental_price_2019_affdf["Primary_MSA"]=rental_price_2019_affdf["Primary_MSA"].astype(str)
rental_price_2019_affdf=rental_price_2019_affdf.set_index("Primary MSA")

#rental_price_2019_affdf['Primary_MSA'] = rental_price_2019_affdf['Primary_MSA'].str.strip()
#rental_price_2019_affdf.dtypes()

rental_price_2019_affdf


,Annual_Rent
Primary MSA,
New York_NY,32189
Los Angeles_CA,30186
Chicago_IL,21155
Dallas_TX,18506
Philadelphia_PA,18942
Houston_TX,17720
Washington_DC,25314
Miami_FL,22838
Atlanta_GA,18247


In [340]:
# Merge 3 datasets (uscities, rentalprice, median income)

firstmerge = pd.merge(longlatdensity_df,median_income_affdf,on='Primary MSA')
firstmerge
affordability_df= pd.merge(firstmerge, rental_price_2019_affdf,on='Primary MSA')
affordability_df.head(10)

,lat,lng,population,density,ranking,firstzip,Median income (dollars),Mean income (dollars),Annual_Rent
Primary MSA,,,,,,,,,
New York_NY,40.6943,-73.9249,18713220.0,10715.0,1,11229,"83,160","122,967",32189
Los Angeles_CA,34.1139,-118.4068,12750807.0,3276.0,1,90291,"77,774","109,583",30186
Chicago_IL,41.8373,-87.6862,8604203.0,4574.0,1,60018,"75,379","104,357",21155
Miami_FL,25.7839,-80.2102,6445545.0,5019.0,1,33129,"60,141","89,694",22838
Dallas_TX,32.7936,-96.7662,5743938.0,1526.0,1,75287,"72,265","100,213",18506
Philadelphia_PA,40.0077,-75.1339,5649300.0,4554.0,1,19154,"74,533","104,703",18942
Houston_TX,29.7863,-95.3889,5464251.0,1399.0,1,77069,"69,193","99,137",17720
Atlanta_GA,33.7627,-84.4224,5449398.0,1441.0,1,30334,"71,742","99,456",18247
Washington_DC,38.9047,-77.0163,5379184.0,4457.0,1,20010,"105,659","138,388",25314


In [341]:
#Calculating affordability

affordability_df['Median income (dollars)'] =affordability_df['Median income (dollars)'].apply(lambda x: float(x.split()[0].replace(',', '')))
affordability_df['Mean income (dollars)']=affordability_df['Mean income (dollars)'].apply(lambda x: float(x.split()[0].replace(',', '')))

affordability_df['Median income (dollars)']=affordability_df['Median income (dollars)'].astype(str).astype(float)
affordability_df['Mean income (dollars)']=affordability_df['Mean income (dollars)'].astype(str).astype(float)

affordability_df['rent_divby_medianinc']=affordability_df['Annual_Rent']/affordability_df['Median income (dollars)']
affordability_df['rent_divby_avginc']=affordability_df['Annual_Rent']/affordability_df['Mean income (dollars)']
affordability_df=affordability_df.sort_values(by=['rent_divby_medianinc'],ascending=False)
affordability_df

,lat,lng,population,density,ranking,firstzip,Median income (dollars),Mean income (dollars),Annual_Rent,rent_divby_medianinc,rent_divby_avginc
Primary MSA,,,,,,,,,,,
Los Angeles_CA,34.1139,-118.4068,12750807.0,3276.0,1,90291,77774.0,109583.0,30186,0.388125,0.275462
New York_NY,40.6943,-73.9249,18713220.0,10715.0,1,11229,83160.0,122967.0,32189,0.387073,0.261769
Miami_FL,25.7839,-80.2102,6445545.0,5019.0,1,33129,60141.0,89694.0,22838,0.379741,0.254621
San Francisco_CA,37.7562,-122.4430,3592294.0,7256.0,1,94130,114696.0,159893.0,38071,0.331930,0.238103
San Diego_CA,32.8312,-117.1225,3220118.0,1686.0,1,92109,83985.0,112786.0,27524,0.327725,0.244037
Riverside_CA,33.9381,-117.3948,2107852.0,1574.0,1,92508,70954.0,91251.0,23117,0.325803,0.253334
Boston_MA,42.3188,-71.0846,4688346.0,5532.0,1,02120,94430.0,129070.0,30409,0.322027,0.235601
Stockton_CA,37.9765,-121.3109,397264.0,1941.0,2,95219,68997.0,92484.0,22099,0.320289,0.238949
Lakeland_FL,28.0556,-81.9545,302284.0,654.0,2,33813,51833.0,67906.0,16132,0.311230,0.237564


In [346]:
# formatting table

formatted_affordabilitydf=affordability_df.style.format({
    'rent_divby_medianinc': '{:,.2%}'.format,
    'rent_divby_avginc': '{:,.2%}'.format,
})

formatted_affordabilitydf

,lat,lng,population,density,ranking,firstzip,Median income (dollars),Mean income (dollars),Annual_Rent,rent_divby_medianinc,rent_divby_avginc
Primary MSA,,,,,,,,,,,
Los Angeles_CA,34.113900,-118.406800,12750807.000000,3276.000000,1,90291,77774.000000,109583.000000,30186,38.81%,27.55%
New York_NY,40.694300,-73.924900,18713220.000000,10715.000000,1,11229,83160.000000,122967.000000,32189,38.71%,26.18%
Miami_FL,25.783900,-80.210200,6445545.000000,5019.000000,1,33129,60141.000000,89694.000000,22838,37.97%,25.46%
San Francisco_CA,37.756200,-122.443000,3592294.000000,7256.000000,1,94130,114696.000000,159893.000000,38071,33.19%,23.81%
San Diego_CA,32.831200,-117.122500,3220118.000000,1686.000000,1,92109,83985.000000,112786.000000,27524,32.77%,24.40%
Riverside_CA,33.938100,-117.394800,2107852.000000,1574.000000,1,92508,70954.000000,91251.000000,23117,32.58%,25.33%
Boston_MA,42.318800,-71.084600,4688346.000000,5532.000000,1,02120,94430.000000,129070.000000,30409,32.20%,23.56%
Stockton_CA,37.976500,-121.310900,397264.000000,1941.000000,2,95219,68997.000000,92484.000000,22099,32.03%,23.89%
Lakeland_FL,28.055600,-81.954500,302284.000000,654.000000,2,33813,51833.000000,67906.000000,16132,31.12%,23.76%
